<a href="https://colab.research.google.com/github/sforseohn/Perfume-Recommender-System/blob/main/Data_Import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joehusseinmama/fragrantica-data")

print("Path to dataset files:", path)

100%|██████████| 1.86G/1.86G [00:17<00:00, 111MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/joehusseinmama/fragrantica-data/versions/1


In [19]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.9 MB/s eta 0:00:00


## Neo4j 연결

In [ ]:
from neo4j import GraphDatabase

# Neo4j 연결 정보
NEO4J_URI = 
NEO4J_USER = 
NEO4J_PASSWORD = 

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


## Cypher 쿼리 실행

In [25]:
import pandas as pd
from neo4j import GraphDatabase
from IPython.display import HTML

# Cypher 쿼리 실행
def run_cypher_query(driver, query: str):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# 사용자 선호 노트
liked_notes = ['vanilla', 'amber', 'sandalwood']
note_filter = ', '.join(f'"{note.lower()}"' for note in liked_notes)

# 겹치는 노트가 많은 순으로 정렬
query = f"""
MATCH (p:Perfume)-[:HAS_NOTE]->(n:Note)
WITH p, COLLECT(n.name) AS allNotes,
     [note IN COLLECT(toLower(n.name)) WHERE note IN [{note_filter}]] AS matchedNotes
WITH p, allNotes, SIZE(matchedNotes) AS matchCount
WHERE matchCount > 0
ORDER BY matchCount DESC
LIMIT 30
RETURN
    p.title AS Title,
    p.designer AS Designer,
    p.description AS Description,
    p.rating AS Rating,
    allNotes AS Notes,
    matchCount AS CommonNotes
"""

# 쿼리 실행
recommendations = run_cypher_query(driver, query)

# DataFrame 변환
df = pd.DataFrame(recommendations)
df['Notes'] = df['Notes'].apply(lambda x: ', '.join(x))
df.index += 1

# 표 출력
HTML(df.to_html(escape=False))


,Title,Designer,Description,Rating,Notes,CommonNotes
1,Dulce de Leche Ganache Parfums for women and men,ganache parfums perfumes and colognes,Dulce de Leche by Ganache Parfums is a Amber Vanilla fragrance for women and men. Dulce de Leche was launched in 2017. The nose behind this fragrance is Jarekhye Covarrubias.,3.67,"vanilla, milk, sandalwood",2
2,Floralique Avon for women,avon perfumes and colognes,"Floralique by Avon is a Floral fragrance for women. Floralique was launched in 1995. Top notes are Pineapple, Gardenia and Violet; middle notes are Jasmine, Lily-of-the-Valley and Rose; base notes are Vanilla, Sandalwood and Musk.",4.00,"rose, jasmine, violet, gardenia, vanilla, sandalwood, musk",2
3,Baby Boy Franck Olivier for men,franck olivier perfumes and colognes,"Baby Boy by Franck Olivier is a Citrus fragrance for men. This is a new fragrance. Baby Boy was launched in 2022. Top notes are Mandarin Orange and Freesia; middle notes are Orange Blossom, Pear and Peony; base notes are Sandalwood, Heliotrope and Raspberry.",2.33,"mandarin orange, pear, raspberry, peony, freesia, orange blossom, sandalwood",1
4,Kanat Angela Ciampagna for women and men,angela ciampagna perfumes and colognes,"Kanat by Angela Ciampagna is a Floral Woody Musk fragrance for women and men. Kanat was launched in 2015. The nose behind this fragrance is Angela Ciampagna. Top notes are Saffron, Bergamot, Apricot, Peach and Black Currant; middle notes are Salt, Magnolia, Cyclamen, Mimosa, Lily and Ylang-Ylang; base notes are Musk, Vanilla and Patchouli.",3.62,"bergamot, peach, magnolia, ylang-ylang, lily, saffron, vanilla, patchouli, musk",1
5,Bleu Royal Princesse Marina De Bourbon for women,princesse marina de bourbon perfumes and colognes,"Bleu Royal by Princesse Marina De Bourbon is a Amber Floral fragrance for women. Bleu Royal was launched in 2012. Top notes are Bergamot, Orange Blossom and Apple; middle notes are Violet, Jasmine and Lotus; base notes are Patchouli, Amber and Sandalwood.",4.01,"bergamot, apple, jasmine, violet, lotus, orange blossom, sandalwood, patchouli",1
6,Dalini Anucci for women,anucci perfumes and colognes,Dalini by Anucci is a Floral fragrance for women. Dalini was launched during the 1990's.,4.33,"jasmine, iris, tuberose, sandalwood, patchouli",1
7,Baroque Fragonard for women,fragonard perfumes and colognes,Baroque by Fragonard is a Floral fragrance for women.,4.19,"bergamot, rose, saffron, ginger, vanilla, guaiac wood, musk",1
8,Shady Maiora Parfum for women and men,maiora parfum perfumes and colognes,"Shady by Maiora Parfum is a fragrance for women and men. Shady was launched in 2019. The nose behind this fragrance is Antonio Gigli. Top notes are Mint, Lavender, Nutmeg and elemi; middle notes are Cypress, Sandalwood, Saffron and Cypriol Oil or Nagarmotha; base notes are Amber, Patchouli, Precious Woods and Musk.",4.22,"mint, nutmeg, saffron, sandalwood, patchouli, musk",1
9,Classique Love Actually Jean Paul Gaultier for women,jean paul gaultier perfumes and colognes,"Classique Love Actually by Jean Paul Gaultier is a Amber Floral fragrance for women. Classique Love Actually was launched in 2011. The nose behind this fragrance is Jacques Cavallier. Top notes are Orange Blossom, Star Anise, Pear, Rose and Mandarin Orange; middle notes are Ylang-Ylang, Iris, Orchid, Plum and Ginger; base notes are Amber, Musk and Vanilla.",4.00,"mandarin orange, pear, plum, rose, iris, orange blossom, ylang-ylang, ginger, vanilla, musk",1
10,Divine Oriflame for women,oriflame perfumes and colognes,"Divine by Oriflame is a Floral fragrance for women. Divine was launched in 2002. The nose behind this fragrance is Jean Jacques. Top notes are Bamboo, Violet, Water Hyacinth, Kiwi and Ivy; middle notes are Orchid, Lily, Freesia, Jasmine and Rose; base notes are White Musk, Sandalwood and Plum.",3.66,"plum, kiwi, rose, jasmine, violet, freesia, lily, sandalwood",1
